In [1]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np

sys.path.append(str(Path("..").resolve()))

from src.constants import raw_data_dir, raw_data_name, processed_data_dir

In [270]:
def lag_return(x, lag=1):
    return ((x - x.shift(lag)) / x.shift(lag)) 

def impute_col(df_sc, col, values):
    df_col = df_sc.loc[:, col].copy()
    df_col = df_col.pct_change(fill_method=None)
    values.update(df_col)
    df_col = pd.Series(values, name = col)
    return df_col

In [271]:
factors = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=1)
factors.set_index("Date", inplace=True)
# factors = factors.apply(lambda x: (x - np.mean(x)) / np.std(x))
categories = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=2)

In [63]:
sc_group = (
    categories.groupby("Subcategory")["Variable"]
    .agg("unique")
    .reset_index()
    .to_dict("list")
)

In [272]:
dfs = []

for idx, sc in enumerate(sc_group["Subcategory"]):
    if sc not in ["Policy Uncertainty", "Sentiment", "Inflation"]:
        cols = sc_group["Variable"][idx]
        df_sc = factors.loc[:, cols]
        mean_returns = df_sc.pct_change(fill_method=None).mean(axis=1)
        for col in cols:
            df_col = impute_col(df_sc, col, mean_returns)
            dfs.append(df_col)

In [273]:
df = pd.concat(dfs, axis=1)

In [297]:
df.apply(np.isinf).sum().sort_values(ascending=False)

US Economic Activity Surprise         120
US Recession Probability in 1 year    120
Global Economic Activity Surprise     120
DM Economic Activity Surprise         120
EM Economic Activity Surprise         120
                                     ... 
IWF                                     0
Min Vol Index                           0
EM currency                             0
Asian currency                          0
VIX                                     0
Length: 97, dtype: int64

In [299]:
def replace_inf(df: pd.DataFrame, factor: int = 3) -> pd.DataFrame:
    def replace_inf_col(col):
        m = np.max(col[~np.isinf(col)])
        return col.replace([np.inf, -np.inf], m * factor)
    return df.apply(replace_inf_col, axis=0)

df = replace_inf(df)

In [300]:
df.apply(np.isinf).sum().sort_values(ascending=False)

S&P 500                                                                                    0
V2X                                                                                        0
GTFRF10YR Corp                                                                             0
GTGBP3MO Corp                                                                              0
GTGBP10YR Corp                                                                             0
                                                                                          ..
MSCI USA Momentum Index                                                                    0
MSCI USA Sector Neutral Quality Index                                                      0
MSCI USA High Dividend Yield Index                                                         0
IWM                                                                                        0
EURJPY Risk Reversal                                                  

In [306]:
df.loc[['2000-07-25', '2000-09-01', '2000-11-28', '2001-03-01',
               '2001-03-26', '2001-03-30', '2001-04-04', '2001-04-24',
               '2001-06-21', '2001-09-04']][["US Economic Activity Surprise"]]

,US Economic Activity Surprise
Date,
2000-07-25,164.0
2000-09-01,164.0
2000-11-28,164.0
2001-03-01,164.0
2001-03-26,164.0
2001-03-30,164.0
2001-04-04,164.0
2001-04-24,164.0
2001-06-21,164.0


In [304]:
df.loc[['2011-09-12', '2012-01-30', '2012-08-22', '2014-06-20',
               '2014-07-08', '2014-07-11', '2014-08-05', '2015-01-06',
               '2015-05-05', '2015-05-07', '2020-12-11', '2021-01-07']][["Copper 1 year carry"]]

,Copper 1 year carry
Date,
2011-09-12,39.0
2012-01-30,-1.0
2012-08-22,39.0
2014-06-20,39.0
2014-07-08,39.0
2014-07-11,39.0
2014-08-05,39.0
2015-01-06,39.0
2015-05-05,39.0


In [261]:
cols = ["S&P 500", "MSCI DM", "MSCI EM", "WTI Crude Oil"
, "S&P GSCI", "Gold", "Global Inflation-linked debt", "iTraxx Europe 5Yr", "iTraxx Crossover 5Yr", "CDX IG 5Yr", 
"CDX HY 5Yr"]

test_raw = factors.iloc[23:27][cols]
test_raw_returns = test_raw.pct_change(fill_method=None)
test_raw_returns

,S&P 500,MSCI DM,MSCI EM,WTI Crude Oil,S&P GSCI,Gold,Global Inflation-linked debt,iTraxx Europe 5Yr,iTraxx Crossover 5Yr,CDX IG 5Yr,CDX HY 5Yr
Date,,,,,,,,,,,
2000-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-07-03,0.010271,0.009699,0.009767,NaN,-0.001323,-0.004076,NaN,NaN,NaN,NaN,NaN
2000-07-05,-0.015862,-0.004202,0.006608,NaN,-0.036942,-0.014219,NaN,NaN,NaN,NaN,NaN
2000-07-06,0.007219,-0.002232,-0.005082,-0.022172,-0.012719,-0.002111,NaN,NaN,NaN,NaN,NaN


In [248]:
mean_returns = test_raw.pct_change(fill_method=None).mean(axis=1)
mean_returns

Date
2000-06-30         NaN
2000-07-03    0.004868
2000-07-05   -0.012924
2000-07-06   -0.006183
dtype: float64

In [249]:
col = test_raw_returns["WTI Crude Oil"]
col

Date
2000-06-30         NaN
2000-07-03         NaN
2000-07-05         NaN
2000-07-06   -0.022172
Name: WTI Crude Oil, dtype: float64

In [250]:
col = col.copy(deep=True)
idx_non_missing = col.notnull()
col[~idx_non_missing] = mean_returns[~idx_non_missing]
col

Date
2000-06-30         NaN
2000-07-03    0.004868
2000-07-05   -0.012924
2000-07-06   -0.022172
Name: WTI Crude Oil, dtype: float64

In [251]:
test_after = df.iloc[24:27][cols]
test_after

,S&P 500,MSCI DM,MSCI EM,WTI Crude Oil,S&P GSCI,Gold,Global Inflation-linked debt,iTraxx Europe 5Yr,iTraxx Crossover 5Yr,CDX IG 5Yr,CDX HY 5Yr
Date,,,,,,,,,,,
2000-07-03,0.010271,0.009699,0.009767,0.004868,-0.001323,-0.004076,0.004868,0.004868,0.004868,0.004868,0.004868
2000-07-05,-0.015862,-0.004202,0.006608,6299.843900,-0.036942,-0.014219,-0.012924,-0.012924,-0.012924,-0.012924,-0.012924
2000-07-06,0.007219,-0.002232,-0.005082,-0.022172,-0.012719,-0.002111,-0.006183,-0.006183,-0.006183,-0.006183,-0.006183


In [ ]:
# drop first and last row (not in targets)
df_all = pd.concat(dfs, axis=1).reset_index().replace([-np.inf, np.inf], np.nan)
# forward fill remaining nas
df_all = df_all.loc[~df_all["Date"].isin(["2000-05-30", "2021-06-30"])].fillna(method="ffill")
# fill S&P 500 VRP, which is has no other subcategory members and has consecutive missing values on top
for col in ["Global Inflation-linked debt", "S&P 500 VRP"]:
    df_all.loc[df_all[col].isna(), col] = df_all.drop(["Date", col], axis=1).loc[df_all[col].isna(), :].mean(axis=1)

df_all.to_csv(processed_data_dir / "df_imputed.csv", index=False)

In [ ]:
df_x = pd.read_csv(processed_data_dir / "df_imputed.csv")
df_x["Date"] = df_x["Date"].astype(str)
df_y = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=3)[["Date", "value_1d_fwd_rel_ret"]]
df_y["Date"] = df_y["Date"].astype(str)
df_y = df_y.query("Date != '2000-05-30'")
df_model_value = df_x.merge(df_y, on = "Date")
df_model_value.head()

,Date,S&P 500,MSCI DM,MSCI EM,WTI Crude Oil,S&P GSCI,Gold,Global Inflation-linked debt,iTraxx Europe 5Yr,iTraxx Crossover 5Yr,...,S&P 500 Price-to-Earnings,P/B,US Value P/E over Growth P/E,US Value P/B over Growth P/B,EquityBond premia,S&P 500 Skew,EURUSD Risk Reversal,USDJPY Risk Reversal,EURJPY Risk Reversal,value_1d_fwd_rel_ret
0,2000-05-31,0.636,1.357,-0.975,4.305,2.188,0.150,-1.798349,1.276833,1.276833,...,-2.556,-2.556,-2.556,-2.556,-2.556,10.280,10.280,10.280,10.280,0.004223
1,2000-06-01,-9.632,-4.054,-0.561,-3.481,-1.406,-0.095,-3.204833,-3.204833,-3.204833,...,-0.206,-0.206,-0.206,-0.206,-0.206,-2.725,-2.725,-2.725,-2.725,-0.003080
2,2000-06-02,-10.749,-9.483,-1.930,-0.702,-0.501,-1.135,-4.083333,-4.083333,-4.083333,...,0.591,0.591,0.591,0.591,0.591,-9.435,-9.435,-9.435,-9.435,0.004965
3,2000-06-05,4.077,-1.211,-0.514,2.121,0.514,-0.591,0.732667,0.732667,0.732667,...,-1.127,-1.127,-1.127,-1.127,-1.127,5.209,5.209,5.209,5.209,-0.005362
4,2000-06-06,3.982,-3.157,0.448,-0.157,0.013,-0.491,0.106333,0.106333,0.106333,...,-0.498,-0.498,-0.498,-0.498,-0.498,-0.928,-0.928,-0.928,-0.928,-0.000852
